In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from pandas.plotting import scatter_matrix
import seaborn as sns

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import warnings
warnings.filterwarnings("ignore")

# 1. Load data

In [ ]:
df_train = pd.read_csv('/kaggle/input/tabular-playground-series-jul-2021/train.csv')
df_test = pd.read_csv('/kaggle/input/tabular-playground-series-jul-2021/test.csv')

In [ ]:
df_train.head()

In [ ]:
df_test.head()

In [ ]:
df_train.shape

In [ ]:
df_test.shape

In [ ]:
df_train.info()

In [ ]:
df_test.info()

In [ ]:
df_test.describe()

In [ ]:
df_train.describe()

# 2. Show data

In [ ]:
columns = df_train.columns
columns

In [ ]:
df_train_copy = df_train.copy(deep=True)
df_train_copy['date_time'] = df_train['date_time'].str.slice(start=-8, stop=-6).astype('float')
df_train_copy['date_time'].value_counts()

In [ ]:
df_test_copy = df_test.copy(deep=True)
df_test_copy['date_time'] = df_test['date_time'].str.slice(start=-8, stop=-6).astype('float')
df_test_copy['date_time'].value_counts()

## 2.1 show corr

In [ ]:
param_columns = columns[:-3].tolist()
param_columns

In [ ]:
corr_matrix = df_train_copy[param_columns + ['target_carbon_monoxide']].corr()
corr_matrix['target_carbon_monoxide'].sort_values(ascending=False)

In [ ]:
scatter_matrix(df_train_copy[param_columns + ['target_carbon_monoxide']], figsize=(50, 50))
plt.tight_layout()
plt.show()

from the last row of the pics we can see some Vertical lines. These lines might be error data.

for example, in absolute_humidity pic.

In [ ]:
corr_matrix = df_train_copy[param_columns + ['target_benzene']].corr()
corr_matrix['target_benzene'].sort_values(ascending=False)

In [ ]:
scatter_matrix(df_train_copy[param_columns + ['target_benzene']], figsize=(50, 50))
plt.tight_layout()
plt.show()

In [ ]:
corr_matrix = df_train_copy[param_columns + ['target_nitrogen_oxides']].corr()
corr_matrix['target_nitrogen_oxides'].sort_values(ascending=False)

In [ ]:
scatter_matrix(df_train_copy[param_columns + ['target_nitrogen_oxides']], figsize=(50, 50))
plt.tight_layout()
plt.show()

## 2.2 exchange data

In [ ]:
plt.rcParams['font.size']=36

In [ ]:
plt.figure(figsize=(50,60))
df_train_copy[param_columns].hist(figsize=(50,40))
plt.show()

## 2.3 log some feature

In [ ]:
df_train_transform = df_train_copy.copy(deep=True)
df_train_transform = np.log(df_train_transform + 1)

plt.figure(figsize=(50,60))
df_train_transform[param_columns].hist(figsize=(50,40))
plt.show()

so we should transform sensor_1\sensor_2\sensor_3\sensor_5

## 2.4 transform data

In [ ]:
df_train_transform = df_train_copy.copy(deep=True)

df_train_transform['sensor_1'] = np.log(df_train_transform['sensor_1'] +1)
df_train_transform['sensor_2'] = np.log(df_train_transform['sensor_2'] +1)
df_train_transform['sensor_3'] = np.log(df_train_transform['sensor_3'] +1)
df_train_transform['sensor_5'] = np.log(df_train_transform['sensor_5'] +1)

df_test_transform = df_test_copy.copy(deep=True)

df_test_transform['sensor_1'] = np.log(df_test_transform['sensor_1'] +1)
df_test_transform['sensor_2'] = np.log(df_test_transform['sensor_2'] +1)
df_test_transform['sensor_3'] = np.log(df_test_transform['sensor_3'] +1)
df_test_transform['sensor_5'] = np.log(df_test_transform['sensor_5'] +1)

## 2.5 find error data

In [ ]:
plt.figure(figsize=(60,40))
num = len(param_columns)
print(num)
col = 3
row = int(num/3) + 1
for i in np.arange(0, num):
    ax = plt.subplot(row, col, i+1)
    sns.boxplot(x=df_train_transform[columns[i]], ax=ax)

plt.tight_layout()
plt.show()

1. absolute_humidity/sensor_1/sensor_2 has one error
2. sensor_4 might has one error

In [ ]:
plt.figure(figsize=(60,40))
num = len(param_columns)
print(num)
col = 3
row = int(num/3) + 1
for i in np.arange(0, num):
    ax = plt.subplot(row, col, i+1)
    sns.scatterplot(x=df_train_transform[columns[i]], y=0, ax=ax)

plt.tight_layout()
plt.show()

In [ ]:
df_train_transform[df_train_transform['absolute_humidity'] == df_train_transform['absolute_humidity'].max()]

In [ ]:
df_train_transform[df_train_transform['sensor_2'] == df_train_transform['sensor_2'].max()]

In [ ]:
df_train_transform[df_train_transform['sensor_1'] == df_train_transform['sensor_1'].min()]

In [ ]:
df_train_transform[df_train_transform['sensor_4'] == df_train_transform['sensor_4'].max()]

In [ ]:
df_train_transform[df_train_transform['sensor_5'] == df_train_transform['sensor_5'].min()]

In [ ]:
df_train_transform[df_train_transform['sensor_5'] == df_train_transform['sensor_5'].max()]

# 3. Find null data

In [ ]:
plt.figure(figsize=(50, 50))
sns.heatmap(df_train_transform.isnull(), cbar=False)
plt.tight_layout()
plt.show()

In [ ]:
Total = df_train_transform.isnull().sum().sort_values(ascending = False)          

# calculate the percentage of missing values
# 'ascending = False' sorts values in the descending order
# the variable with highest percentage of missing values will appear first
Percent = ((Total*100)/df_train_transform.isnull().count()).sort_values(ascending = False)   

# concat the 'Total' and 'Percent' columns using 'concat' function
# 'keys' is the list of column names
# 'axis = 1' concats along the columns
missing_data = pd.concat([Total, Percent], axis = 1, keys = ['Total', 'Percentage of Missing Values'])    
missing_data[missing_data['Total']>0]

In [ ]:
Total = df_test_transform.isnull().sum().sort_values(ascending = False)          

# calculate the percentage of missing values
# 'ascending = False' sorts values in the descending order
# the variable with highest percentage of missing values will appear first
Percent = ((Total*100)/df_test_transform.isnull().count()).sort_values(ascending = False)   

# concat the 'Total' and 'Percent' columns using 'concat' function
# 'keys' is the list of column names
# 'axis = 1' concats along the columns
missing_data = pd.concat([Total, Percent], axis = 1, keys = ['Total', 'Percentage of Missing Values'])    
missing_data[missing_data['Total']>0]

Great! There is no null data.

# 4. Compare train and test data

In [ ]:
plt.figure(figsize=(60,60))
num = len(param_columns)
print(num)
col = 3
row = int(num/3) + 1
for i in np.arange(0, num):
    ax = plt.subplot(row, col, i+1)
    sns.kdeplot(x=df_train_transform[param_columns[i]], ax=ax, label='train')
    sns.kdeplot(x=df_test_transform[param_columns[i]], ax=ax, label='test')

plt.tight_layout()
plt.legend()
plt.show()

# 5. transform target and scale features

In [ ]:
df_train_transform['target_carbon_monoxide'] = np.log(df_train_transform['target_carbon_monoxide'] +1)
df_train_transform['target_benzene'] = np.log(df_train_transform['target_benzene'] +1)
df_train_transform['target_nitrogen_oxides'] = np.log(df_train_transform['target_nitrogen_oxides'] +1)

In [ ]:
df_train_transform[columns[-3:]].hist(figsize=(50,40))

In [ ]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler
mmS = MinMaxScaler()
ss = StandardScaler()

all_train_x = df_train_transform[columns[:-3]].values
all_train_y = df_train_transform[columns[-3:]].values

all_test_x = df_test_transform.values

all_train_x = mmS.fit_transform(all_train_x)
all_train_x = ss.fit_transform(all_train_x)

all_test_x = mmS.transform(all_test_x)
all_test_x = ss.transform(all_test_x)

In [ ]:
all_test_x[0]

In [ ]:
all_train_x[0]

# 6. test different models

## split data

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(all_train_x, all_train_y, test_size = 0.20, random_state = 10)

In [ ]:
for obj in [X_train, X_test, y_train, y_test]:
    print(obj.shape)

## RMSLE

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
def rmsle(y_pred, y_test):
    res = []
    for i in range(y_test.shape[1]):
        res.append(np.sqrt(mean_squared_error(y_test[:,i],y_pred[:,i])))
    
    return res

In [ ]:
score_card = pd.DataFrame(columns=['Model', 'RMSLE'])


def update_score_card(model_name, rmsle):
    
    # assign 'score_card' as global variable
    global score_card

    # append the results to the dataframe 'score_card'
    # 'ignore_index = True' do not consider the index labels
    score_card = score_card.append({'Model': model_name,
                                    'RMSLE' : rmsle}, 
                                    ignore_index = True)
    return(score_card)

## 6.1 xgboost

In [ ]:
from sklearn.multioutput import MultiOutputRegressor
from xgboost import XGBRegressor
xg_model = MultiOutputRegressor(XGBRegressor())
xg_model.fit(X_train, y_train)

In [ ]:
y_test_predict = xg_model.predict(X_test)

In [ ]:
y_test_predict[0]

In [ ]:
y_test[0]

In [ ]:
xg_error = rmsle(y_test_predict, y_test)
update_score_card('xgboost', xg_error)
xg_error

## 6.2 LightGBM

In [ ]:
from lightgbm import LGBMRegressor
gbm_model = MultiOutputRegressor(LGBMRegressor())
gbm_model.fit(X_train, y_train)

In [ ]:
y_test_predict = gbm_model.predict(X_test)

In [ ]:
gbm_error = rmsle(y_test_predict, y_test)
update_score_card('lightgbm', gbm_error)
gbm_error

## 6.3 catboost

In [ ]:
from catboost import CatBoostRegressor
cat_model = MultiOutputRegressor(CatBoostRegressor())
cat_model.fit(X_train, y_train, verbose=False)

In [ ]:
y_test_predict = cat_model.predict(X_test)

In [ ]:
cat_error = rmsle(y_test_predict, y_test)
update_score_card('catboost', cat_error)
cat_error

So, catboost is the best GBDT model.

## 6.4 SVR

In [ ]:
# from sklearn.svm import SVR
# svr_model = MultiOutputRegressor(SVR(kernel = 'linear'))
# svr_model.fit(X_train, y_train)

In [ ]:
# y_test_predict = svr_model.predict(X_test)

In [ ]:
# svr_error = rmsle(y_test_predict, y_test.values)
# update_score_card('svr', svr_error)
# svr_error

## 6.5 LinearRegression

In [ ]:
from sklearn.linear_model import Lasso, Ridge

In [ ]:
lasso_model = MultiOutputRegressor(Lasso())
lasso_model.fit(X_train, y_train)

In [ ]:
y_test_predict = lasso_model.predict(X_test)

In [ ]:
svr_error = rmsle(y_test_predict, y_test)
update_score_card('svr', svr_error)
svr_error

In [ ]:
ridge_model = MultiOutputRegressor(Ridge())
ridge_model.fit(X_train, y_train)

In [ ]:
y_test_predict = ridge_model.predict(X_test)

In [ ]:
ridge_error = rmsle(y_test_predict, y_test)
model_res = update_score_card('ridge', ridge_error)
ridge_error

In [ ]:
model_res

# 7. Tuning

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
def print_best_score(gsearch, param_test):
    print("Best score: %0.3f" % gsearch.best_score_)
    print("Best parameters set:")
    best_parameters = gsearch.best_estimator_.get_params()
    for param_name in sorted(param_test.keys()):
        print("\t%s: %r" % (param_name, best_parameters[param_name]))

In [ ]:
params = {'depth': [2, 6, 8],
          'learning_rate' : [0.001, 0.01, 0.1],
          'l2_leaf_reg': [2, 4, 6, 8],
          'iterations': [1000, 5000, 10000]
         }

In [ ]:
estimator1 = CatBoostRegressor(verbose=1000, loss_function='RMSE')
cbt_model1 = GridSearchCV(estimator1, param_grid = params, cv = 5)
cbt_model1.fit(X_train,y_train[:,0],eval_set=(X_test,y_test[:,0]))
print_best_score(cbt_model1, params)

In [ ]:
cbt_model1.cv_results_

In [ ]:
estimator2 = CatBoostRegressor(verbose=1000,loss_function='RMSE')
cbt_model2 = GridSearchCV(estimator2, param_grid = params, cv = 5)
cbt_model2.fit(X_train,y_train[:,1],eval_set=(X_test,y_test[:,1]))
print_best_score(cbt_model2, params)

In [ ]:
cbt_model2.cv_results_

In [ ]:
estimator3 = CatBoostRegressor(verbose=1000,loss_function='RMSE')
cbt_model3 = GridSearchCV(estimator3, param_grid = params, cv = 5)
cbt_model3.fit(X_train,y_train[:,2],eval_set=(X_test,y_test[:,2]))
print_best_score(cbt_model3, params)

# 7. predict

In [ ]:
res_arr = []

In [ ]:
best_model1=cbt_model1.best_estimator_
best_model1.fit(all_train_x, all_train_y[:,0])
predict_y1=best_model1.predict(all_test_x)
res_arr.append(np.exp(predict_y1)-1)

In [ ]:
best_model2=cbt_model2.best_estimator_
best_model2.fit(all_train_x, all_train_y[:,1])
predict_y2=best_model2.predict(all_test_x)
res_arr.append(np.exp(predict_y2)-1)

In [ ]:
best_model3=cbt_model3.best_estimator_
best_model3.fit(all_train_x, all_train_y[:,2])
predict_y3=best_model3.predict(all_test_x)
res_arr.append(np.exp(predict_y3)-1)

In [ ]:
res_arr_t = np.array(res_arr).T
res_arr_t.shape

In [ ]:
res = pd.DataFrame(data=res_arr_t, columns=['target_carbon_monoxide', 'target_benzene', 'target_nitrogen_oxides'])
res['date_time'] = df_test['date_time']
res.to_csv('res.csv', index=0)